In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation, metrics
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CustomJS, HoverTool
from bokeh.io import output_notebook, push_notebook
from bokeh.layouts import gridplot, widgetbox, layout
from bokeh.models.widgets import Select
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6, Spectral11
from bokeh.models.widgets import Select

from pipelines import *

%matplotlib inline

In [4]:
training_path = '/home/alvin/!Final_Project/training_with_tokens.xlsx'
testing_path = '/home/alvin/!Final_Project/testing_with_tokens.xlsx'

embedding_dim = 10
top_n_token = 10

print('Load data...')
df_train = load_data(training_path)

Load data...


In [5]:
df_train.head()

,class,tokens,sentence
0,2,"[合晟资产, 专注, 股票, 债券, 二级市场, 投资, 合格, 投资者, 资产, 管理, ...",合晟资产 专注 股票 债券 二级市场 投资 合格 投资者 资产 管理 企业 业务范围 资产 ...
1,2,"[中, 小微企业, 个体, 工商户, 农户, 贷款, 设立, 发生, 变化, UNKNOWN]",中 小微企业 个体 工商户 农户 贷款 设立 发生 变化 UNKNOWN
2,1,"[立足于, 商业地产, 商业地产, 开发, 销售, 运营, 全产业链, 一整套, 增值, 业...",立足于 商业地产 商业地产 开发 销售 运营 全产业链 一整套 增值 业务 覆盖 商业 定位...
3,2,"[工商管理部门, 核准, 经营范围, 投资, 咨询, 经济, 信息, 咨询, 企业管理, 咨...",工商管理部门 核准 经营范围 投资 咨询 经济 信息 咨询 企业管理 咨询 品牌 推广 策划...
4,2,"[中国, 境内, 港, 澳, 台, 保险代理, 销售, 研究, 能力, 专业化, 能力, 团...",中国 境内 港 澳 台 保险代理 销售 研究 能力 专业化 能力 团体 个人保险 受众 投保...


In [8]:
df_x = df_train.drop(['class'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(df_x, df_train['class'], test_size=0.2, random_state=11, stratify=df_train['class'])

In [10]:
x_train.head()

,tokens,sentence
3839,"[国内, 信息安全, 整体, 解决方案, 提供商, 专注, 信息安全, 领域, 研发, 国际...",国内 信息安全 整体 解决方案 提供商 专注 信息安全 领域 研发 国际 先进 网络 信息安...
910,"[海水淡化设备, 凝汽器, 汽轮机, 辅机, 设备, 锅炉, 辅机, 设备, 设计, 制造,...",海水淡化设备 凝汽器 汽轮机 辅机 设备 锅炉 辅机 设备 设计 制造 销售 安装 维护 技...
4702,"[工商部门, 核准, 经营范围, 许可经营项目, 经营项目, 计算机软件, 网络, 技术, ...",工商部门 核准 经营范围 许可经营项目 经营项目 计算机软件 网络 技术 开发 技术 计算机...
4245,"[成立, 1997年, 集, 蜜柚, 种植, 加工, 销售, 一体, 广东省, 重点, 农业...",成立 1997年 集 蜜柚 种植 加工 销售 一体 广东省 重点 农业 龙头企业 广东省 扶...
1706,"[杭州泛远国际物流有限公司, 始创, 2004年, 主营, 国际海运, 国际, 空运, 国际...",杭州泛远国际物流有限公司 始创 2004年 主营 国际海运 国际 空运 国际 快件 代理 业...


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

class TokensPicker(BaseEstimator, TransformerMixin):
    
    def __init__(self, top_n=10, vectors):
        self.top_n = top_n
        self.vectors = vectors
        
    def fit(self, X, y=None):
        return self
    
    def get_top_tokens_in_doc(df, vectors, features, row_id, top_n=25):
        row = np.squeeze(vectors[row_id].toarray())
        tokens = df.loc[row_id]['tokens']
        token_length = len(tokens)
    #     print('Token length: ', str(token_length))
        token_values = {}
        for i in range(token_length):
            # Get tfidf score for each token
            token_name = tokens[i]
            try:
                if token_name in vectorizer.vocabulary_:
                    token_index = vectorizer.vocabulary_[token_name]
                    token_value = row[token_index]
                else:
                    token_value = 0
            except:
                print("Exception: ", str(row_id))
            token_values[token_name] = token_value
        # Sort the tokens by tfidf values
        sorted_tokens = sorted(token_values.items(), key=operator.itemgetter(1), reverse=True)
    #     print(sorted_tokens)
        # Get the most weighted tokens
        top_tokens = []
        padding_count = 0
    #     print("Sorted tokens length: ", str(len(sorted_tokens)))
        if len(sorted_tokens) < top_n:
            padding_count = top_n - len(sorted_tokens)
            for i in range(len(sorted_tokens)):
                top_tokens.append(sorted_tokens[i][0])
        else:
            for i in range(top_n):
                top_tokens.append(sorted_tokens[i][0])
        for i in range(padding_count):
            top_tokens.append('UNKNOWN')
        return top_tokens
    
    def transform(self, X, y=None):
        